#### City Bike Analytics Workbook

In [1]:
import pandas as pd

In [2]:
# specify data path
jan = "../Tableau-CitiBike-Visualization/Data/JC-202101-citibike-tripdata.csv"
feb = "../Tableau-CitiBike-Visualization/Data/JC-202102-citibike-tripdata.csv"
mar = "../Tableau-CitiBike-Visualization/Data/JC-202103-citibike-tripdata.csv"

# read csv file into jupyter notebook; read datetime columns correctly by using 'parse_dates' 
jan_data = pd.read_csv(jan, parse_dates=['starttime', 'stoptime'])
feb_data = pd.read_csv(feb, parse_dates=['started_at', 'ended_at'])
mar_data = pd.read_csv(mar, parse_dates=['started_at', 'ended_at'])

In [3]:
# Splitting timestamp columns into separate date and time columns

jan_data['start_date'] = pd.to_datetime(jan_data['starttime']).dt.date
jan_data['start_time'] = pd.to_datetime(jan_data['starttime']).dt.time
jan_data['end_date'] = pd.to_datetime(jan_data['stoptime']).dt.date
jan_data['end_time'] = pd.to_datetime(jan_data['stoptime']).dt.time

feb_data['start_date'] = pd.to_datetime(feb_data['started_at']).dt.date
feb_data['start_time'] = pd.to_datetime(feb_data['started_at']).dt.time
feb_data['end_date'] = pd.to_datetime(feb_data['ended_at']).dt.date
feb_data['end_time'] = pd.to_datetime(feb_data['ended_at']).dt.time
mar_data['start_date'] = pd.to_datetime(mar_data['started_at']).dt.date
mar_data['start_time'] = pd.to_datetime(mar_data['started_at']).dt.time
mar_data['end_date'] = pd.to_datetime(mar_data['ended_at']).dt.date
mar_data['end_time'] = pd.to_datetime(mar_data['ended_at']).dt.time

In [4]:
feb_data.head(1)

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,start_date,start_time,end_date,end_time
0,121DD7DD23CB1335,docked_bike,2021-02-03 23:11:28,2021-02-03 23:18:28,Hoboken Ave at Monmouth St,JC105,Christ Hospital,JC034,40.735208,-74.046964,40.734786,-74.050444,member,2021-02-03,23:11:28,2021-02-03,23:18:28


In [5]:
# rename columns
new_jan_df = jan_data
new_jan_df.rename(columns={"tripduration":"trip_duration",
                           "starttime":"started_at",
                           "stoptime":"ended_at",
                           "start station id":"start_station_id",
                           "end station id":"end_station_id",
                           "start station name":"start_station_name",
                           "end station name":"end_station_name",
                           "start station latitude":"start_lat",
                           "end station latitude":"end_lat",
                           "start station longitude":"start_lng",
                           "end station longitude":"end_lng",
                           "start station name":"start_station_name",
                           "end station name":"end_station_name"
                        }, inplace=True)

In [6]:
# drop columns to harmonize columns with feb and mar datasets
new_jan_df.drop(["trip_duration", "usertype", "bikeid", "birth year", "gender"], axis=1, inplace=True)

In [7]:
# combine feb & mar df into one df since the df heads are the same
new_feb_mar_df = pd.concat([feb_data, mar_data])

In [8]:
# drop columns to harmonize columns with jan dataset
new_feb_mar_df.drop(["ride_id", "rideable_type", "member_casual"], axis=1, inplace=True)

In [9]:
# merge two dataframes together
merge_df = pd.concat([new_jan_df, new_feb_mar_df], axis=0, sort=False)

In [10]:
# Add ride id column to uniquely identify each ride
merge_df["rideid"] = merge_df["start_station_id"].map(str) + "_" + merge_df["end_station_id"].map(str)
merge_df.head(1)

,started_at,ended_at,start_station_id,start_station_name,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,start_date,start_time,end_date,end_time,rideid
0,2021-01-01 00:03:35.510,2021-01-01 00:08:01.777,3273,Manila & 1st,40.721651,-74.042884,3209,Brunswick St,40.724176,-74.050656,2021-01-01,00:03:35.510000,2021-01-01,00:08:01.777000,3273_3209


In [11]:
merge_df = merge_df.astype({"start_date": str})

# break out year
merge_df['trip_year'] = merge_df.start_date.str.split("-", expand=True)[0]
merge_df.head(1)

,started_at,ended_at,start_station_id,start_station_name,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,start_date,start_time,end_date,end_time,rideid,trip_year
0,2021-01-01 00:03:35.510,2021-01-01 00:08:01.777,3273,Manila & 1st,40.721651,-74.042884,3209,Brunswick St,40.724176,-74.050656,2021-01-01,00:03:35.510000,2021-01-01,00:08:01.777000,3273_3209,2021


In [12]:
# change column dtype to string
merge_df = merge_df.astype({"start_date": str})

# break out month
merge_df['trip_month'] = merge_df.start_date.str.split("-", expand=True)[1]

In [13]:
# merge_df['end_time'] = pd.to_datetime(merge_df['start_time'], errors='coerce')
# merge_df['start_time'] = pd.to_datetime(merge_df['start_time'], errors='coerce')

In [14]:
# calculated travel time between bike stations
# merge_df['travel_time (min)'] = merge_df['end_time'].dt.minute - merge_df['start_time'].dt.minute

merge_df.head(2)

,started_at,ended_at,start_station_id,start_station_name,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,start_date,start_time,end_date,end_time,rideid,trip_year,trip_month
0,2021-01-01 00:03:35.510,2021-01-01 00:08:01.777,3273,Manila & 1st,40.721651,-74.042884,3209,Brunswick St,40.724176,-74.050656,2021-01-01,00:03:35.510000,2021-01-01,00:08:01.777000,3273_3209,2021,01
1,2021-01-01 00:23:32.925,2021-01-01 00:49:16.083,3681,Grand St,40.715178,-74.037683,3213,Van Vorst Park,40.718489,-74.047727,2021-01-01,00:23:32.925000,2021-01-01,00:49:16.083000,3681_3213,2021,01


In [15]:
merge_df["trip_month"].replace({"01": "January", "02": "February", "03": "March"}, inplace=True)
merge_df.head(1)

,started_at,ended_at,start_station_id,start_station_name,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,start_date,start_time,end_date,end_time,rideid,trip_year,trip_month
0,2021-01-01 00:03:35.510,2021-01-01 00:08:01.777,3273,Manila & 1st,40.721651,-74.042884,3209,Brunswick St,40.724176,-74.050656,2021-01-01,00:03:35.510000,2021-01-01,00:08:01.777000,3273_3209,2021,January


In [16]:
# drop rows that have blank or NaN values
merge_df.dropna(subset=["end_lat", "end_lng", "end_station_name"], inplace = True)

# defective bikes are likely to be returned to the same station
# drop rows where both start and end stations are the same
merge_df = merge_df[merge_df["start_station_name"] != merge_df["end_station_name"]]

###### Calculate distance of each ride point a to b.

In [17]:
from math import sin, cos, sqrt, atan2, radians

df = merge_df

# For lat and lon, convert degrees to radians
lat1 = [radians(lat) for lat in df["start_lat"]]
lon1 = [radians(lon) for lon in df["start_lng"]]
lat2 = [radians(lat) for lat in df["end_lat"]]
lon2 = [radians(lon) for lon in df["end_lng"]]

# Convert lists into series
lat1 = pd.Series(lat1)
lon1 = pd.Series(lon1)
lat2 = pd.Series(lat2)
lon2 = pd.Series(lon2)

# Calculate difference between each set of latitude and longitude
dlat = lat2 - lat1
dlon = lon2 - lon1

In [18]:
# Approximate radius of Earth in km
R = 6373.0

# Empty list to store trip distances
distance = []

for i in range(0, len(lat1)):
    
    a = sin(dlat[i] / 2)**2 + cos(lat1[i]) * cos(lat2[i]) * sin(dlon[i] / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    # Get distance and convert km to miles
    miles = (R * c) * .6214
    
    # Append miles travel to 'distance' list
    distance.append(miles)

In [19]:
# Add trip distance as new column to data frame
df.loc[:, "trip_distance (mi)"] = distance

# df["tripdistance (mi)"] = distance
df

,started_at,ended_at,start_station_id,start_station_name,start_lat,start_lng,end_station_id,end_station_name,end_lat,end_lng,start_date,start_time,end_date,end_time,rideid,trip_year,trip_month,trip_distance (mi)
0,2021-01-01 00:03:35.510,2021-01-01 00:08:01.777,3273,Manila & 1st,40.721651,-74.042884,3209,Brunswick St,40.724176,-74.050656,2021-01-01,00:03:35.510000,2021-01-01,00:08:01.777000,3273_3209,2021,January,0.442985
1,2021-01-01 00:23:32.925,2021-01-01 00:49:16.083,3681,Grand St,40.715178,-74.037683,3213,Van Vorst Park,40.718489,-74.047727,2021-01-01,00:23:32.925000,2021-01-01,00:49:16.083000,3681_3213,2021,January,0.573768
2,2021-01-01 00:23:50.794,2021-01-01 00:48:12.566,3681,Grand St,40.715178,-74.037683,3213,Van Vorst Park,40.718489,-74.047727,2021-01-01,00:23:50.794000,2021-01-01,00:48:12.566000,3681_3213,2021,January,0.573768
3,2021-01-01 00:31:09.077,2021-01-01 00:44:22.943,3185,City Hall,40.717733,-74.043845,3199,Newport Pkwy,40.728745,-74.032108,2021-01-01,00:31:09.077000,2021-01-01,00:44:22.943000,3185_3199,2021,January,0.978435
4,2021-01-01 00:35:52.190,2021-01-01 00:45:48.774,3639,Harborside,40.719252,-74.034234,3209,Brunswick St,40.724176,-74.050656,2021-01-01,00:35:52.190000,2021-01-01,00:45:48.774000,3639_3209,2021,January,0.925164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17118,2021-03-29 19:30:20.000,2021-03-29 19:38:10.000,JC104,Harborside,40.719251,-74.034234,JC072,Morris Canal,40.712419,-74.038526,2021-03-29,19:30:20,2021-03-29,19:38:10,JC104_JC072,2021,March,0.523016
17119,2021-03-12 18:49:06.000,2021-03-12 18:54:35.000,JC104,Harborside,40.719251,-74.034234,JC072,Morris Canal,40.712419,-74.038526,2021-03-12,18:49:06,2021-03-12,18:54:35,JC104_JC072,2021,March,0.523016
17120,2021-03-28 12:29:15.000,2021-03-28 12:34:40.000,JC104,Harborside,40.719251,-74.034234,JC032,Newark Ave,40.721525,-74.046305,2021-03-28,12:29:15,2021-03-28,12:34:40,JC104_JC032,2021,March,0.651558
17121,2021-03-23 13:03:35.000,2021-03-23 13:10:25.000,JC104,Harborside,40.719251,-74.034234,JC032,Newark Ave,40.721525,-74.046305,2021-03-23,13:03:35,2021-03-23,13:10:25,JC104_JC032,2021,March,0.651558


In [22]:
df.to_csv("../Tableau-CitiBike-Visualization/Output/upload_to_tableau.csv")

In [20]:
trips_df = df.groupby(["start_station_name","end_station_name"]).size().reset_index(name = "Number_of_Trips")

trips_df.to_csv("../Tableau-CitiBike-Visualization/Output.csv")

,start_station_name,end_station_name,Number_of_Trips
0,5 Corners Library,Baldwin at Montgomery,3
1,5 Corners Library,Bergen Ave,1
2,5 Corners Library,Brunswick & 6th,2
3,5 Corners Library,Brunswick St,10
4,5 Corners Library,Christ Hospital,5
...,...,...,...
1998,York St,Paulus Hook,3
1999,York St,Sip Ave,1
2000,York St,Van Vorst Park,13
2001,York St,Warren St,7
